In [1]:
import numpy as np
from keras import layers
from keras.layers import Input,Dense,Activation,ZeroPadding2D,BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,GlobalMaxPooling2D,Dropout,GlobalAveragePooling2D
from keras.utils import np_utils,print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

Using TensorFlow backend.


In [2]:
data=pd.read_csv("D:\Datasets\DevanagariHandwrittenCharacterDataset\data.csv")
dataset=np.array(data)
np.random.shuffle(dataset)
X=dataset
Y=dataset
X=X[:,0:1024]
Y=Y[:,1024]

X_train=X[0:70000,:]
X_train=X_train/255
X_test=X[70000:72001,:]
X_test=X_test/255

Y=Y.reshape(Y.shape[0],1)
Y_train=Y[0:70000,:]
Y_train=Y_train.T
Y_test=Y[70000:72001,:]
Y_test=Y_test.T

print("....TRAINING AND TESTING DATASET SEPERATED....")

....TRAINING AND TESTING DATASET SEPERATED....


In [3]:
image_x=32
image_y=32

train_y=np_utils.to_categorical(Y_train)
test_y=np_utils.to_categorical(Y_test)
train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
X_train=X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test=X_test.reshape(X_test.shape[0],image_x,image_y,1)

print("X_train shape:",X_train.shape)
print("Y_train shape:",train_y.shape)

X_train shape: (70000, 32, 32, 1)
Y_train shape: (70000, 37)


In [4]:
def keras_model(image_x,image_y):
    num_of_classes=37
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devnagari.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    
    return model,callbacks_list

In [5]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(X_train,train_y,validation_data=(X_test,test_y),epochs=5,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(X_test,test_y,verbose=0)
print('CNN ERROR=%.2f%%'%(100-scores[1] *100))
print_summary(model)
model.save("devnagari.h5")

Train on 70000 samples, validate on 2000 samples
Epoch 1/5
70000/70000 [==============================] - 75s 1ms/step - loss: 0.6530 - accuracy: 0.8130 - val_loss: 0.2730 - val_accuracy: 0.9075
Epoch 2/5
  128/70000 [..............................] - ETA: 1:17 - loss: 0.1848 - accuracy: 0.9297

C:\Users\KIIT\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


70000/70000 [==============================] - 96s 1ms/step - loss: 0.1738 - accuracy: 0.9461 - val_loss: 0.1735 - val_accuracy: 0.9535
Epoch 3/5
70000/70000 [==============================] - 130s 2ms/step - loss: 0.1137 - accuracy: 0.9651 - val_loss: 0.1454 - val_accuracy: 0.9615
Epoch 4/5
70000/70000 [==============================] - 130s 2ms/step - loss: 0.0802 - accuracy: 0.9749 - val_loss: 0.1304 - val_accuracy: 0.9590
Epoch 5/5
70000/70000 [==============================] - 132s 2ms/step - loss: 0.0624 - accuracy: 0.9802 - val_loss: 0.1123 - val_accuracy: 0.9670
CNN ERROR=3.30%
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_______________________________________________________